In [1]:
from utilities.download import download_and_read_json_xz

# Example usage
url = "http://dadosabertos.c3sl.ufpr.br/curitibaurbs/2023_11_24_veiculos.json.xz"
df = download_and_read_json_xz(url, lines=True)

In [2]:
from utilities.coordinates import format_coord

df['coords'] = df.apply(lambda row: format_coord(row['LAT'], row['LON']), axis=1)
df.drop(['LAT', 'LON'], axis=1, inplace=True)

In [3]:
df

,VEIC,DTHR,COD_LINHA,DATE,coords
0,BA001,23/11/2023 08:41:42,231,2023-11-24,"(-49.223578, -25.376803)"
1,BA001,23/11/2023 08:41:37,231,2023-11-24,"(-49.223948, -25.376796)"
2,BA001,23/11/2023 08:41:12,231,2023-11-24,"(-49.224335, -25.376888)"
3,BA001,23/11/2023 08:41:02,231,2023-11-24,"(-49.224515, -25.376855)"
4,BA001,23/11/2023 08:40:57,231,2023-11-24,"(-49.224751, -25.376883)"
...,...,...,...,...,...
3122906,XY044,23/11/2023 06:29:47,205,2023-11-24,"(-49.270195, -25.429005)"
3122907,XY044,23/11/2023 06:29:42,205,2023-11-24,"(-49.270381, -25.428828)"
3122908,XY044,23/11/2023 06:29:07,205,2023-11-24,"(-49.270653, -25.428561)"
3122909,XY044,23/11/2023 06:25:22,205,2023-11-24,"(-49.27093, -25.4284)"


In [4]:
from utilities.elastic import get_elastic_client

es = get_elastic_client()

In [5]:
index_name = 'veiculos'

index_settings = {
"settings": {
    "number_of_shards": 1,
    "number_of_replicas": 1
  },
  "mappings": {
    "properties": {
      "VEIC": { "type": "text" },
      "DTHR": { 
        "type": "date", 
        "format": "dd/MM/yyyy HH:mm:ss"
      },
      "COD_LINHA": { "type": "integer" },
      "date": { "type": "date", "format": "yyyy-MM-dd" },
      "coords": { "type": "geo_point" }
    }
  }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_settings)

In [7]:
from utilities.elastic import bulk_insert

bulk_insert(es, df, index_name)

BulkIndexError: 3 document(s) failed to index.